In [ ]:
!pip install pymysql
import pymysql


host = 'dhauz-instance.cutloqirhpd7.us-east-1.rds.amazonaws.com'
port = 3306
username = 'candidate_user'
password = 'D3@bGh664%$1VHv*'
database = 'db_hiring_test'

Fase 1 – Análise sobre as transações:

In [ ]:
# 1-Qual é o endereço (carteira) com maior volume de transações enviadas?
# R: Carteira A-83
# Observação: "volume de transações" foi considerado como a soma do valor enviado.

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        )
        SELECT AddressOrigin, SUM(TotalSent) FROM ranked_transactions
        WHERE rn = 1 AND status = 'Confirmed'
        GROUP BY AddressOrigin ORDER BY SUM(TotalSent) DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 2-Qual é o dia do mês com maior volume de transações realizadas?
# R: Dia 07
# Observação: para esse caso optou-se por desconsiderar as transações negadas, considerando somente as transações confirmadas e pendentes.

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        )
        SELECT DAY(SentDate), SUM(TotalSent) FROM ranked_transactions
        WHERE rn = 1 AND status <> 'Denied'
        GROUP BY DAY(SentDate) ORDER BY SUM(TotalSent) DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 3-Em qual dia da semana geralmente mais transações são realizadas?
# R: Quinta-feira
# Observação: para esse caso optou-se por desconsiderar as transações negadas, considerando somente as transações confirmadas e pendentes.

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        )
        SELECT DAYOFWEEK(SentDate), AVG(TotalSent) FROM ranked_transactions
        WHERE rn = 1 AND status <> 'Denied'
        GROUP BY DAYOFWEEK(SentDate) ORDER BY AVG(TotalSent) DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 4-Quais transações possuem condições atípicas e precisam ser validadas com o time responsável pela disponibilização dos dados?
# R: Transações cuja carteira de origem e destino são iguais, seria atípico pois esperava-se que a transação fosse o envio de valor de
# uma carteira para outra. Outras questões atípicas podemos citar valores vazios para TotalSent bem como valores entre parênteses

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        SELECT * FROM ranked_transactions
        WHERE AddressOrigin = AddressDestination;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 5- Qual a carteira com o maior saldo final? (considere que todas as carteiras estejam zeradas no
# início das análises e que seja possível existir saldo negativo).
# R: Foi criado um SELECT que lista todas as carteiras atuais sem duplicidade e seus atuais valores enviados e recebidos para que
# seja realizado o cálculo de valor atual a partir dos enviados e recebidos.
# De acordo com a query abaixo a carteira com maior saldo final seria a carteira A-30

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        ), 
        sent_values AS 
        (
            SELECT AddressOrigin AS Address, SUM(TotalSent) AS TotalSent FROM ranked_transactions
            WHERE rn = 1 AND status = 'Confirmed'
            GROUP BY AddressOrigin
        ),
        received_value AS
        (
            SELECT AddressDestination AS Address, SUM(TotalSent) AS TotalReceived FROM ranked_transactions
            WHERE rn = 1 AND status = 'Confirmed'
            GROUP BY AddressDestination
        ),
        addresses AS
        (
            SELECT Address FROM sent_values
            UNION
            SELECT Address FROM received_value 
        )
        SELECT a.Address, rv.TotalReceived, sv.TotalSent, (rv.TotalReceived - sv.TotalSent) AS CurrentAddressValue
            FROM addresses a
        LEFT JOIN received_value rv ON a.Address = rv.Address
        LEFT JOIN sent_values sv ON a.Address = sv.Address
        ORDER BY CurrentAddressValue DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

Fase 2 – Análise sobre as taxas:

In [ ]:
# 1-Considerando que a carteira origem é responsável por pagar as taxas de envio, qual carteira
# seria responsável pelo maior pagamento de taxas em janeiro de 2021?
# R: Carteira A-99.
# Obs: estamos desconsiderando valores entre parênteses que devems er avalaidos com a área de negócios

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        ),
        transactions AS
        (
            SELECT AddressOrigin, 
                CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) AS FixedTotalSent, 
                `fee-percentage`
            FROM ranked_transactions tab
            LEFT JOIN db_hiring_test.raw_transactions_fee fee 
                ON CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    >= CAST(fee.`range-start` AS DECIMAL(10,2)) 
                AND CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    <= CAST(fee.`range-end` AS DECIMAL(10,2))
            WHERE tab.rn = 1 AND tab.status = 'Confirmed' AND SentDate >= '2021-01-01' AND SentDate < '2021-02-01'
        )
        SELECT AddressOrigin, SUM(FixedTotalSent * `fee-percentage` / 100) AS fee  FROM transactions
        GROUP BY AddressOrigin
        ORDER BY fee DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 2- E em fevereiro de 2021?
# R: Carteira A-29

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        ),
        transactions AS
        (
            SELECT AddressOrigin, 
                CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) AS FixedTotalSent, 
                `fee-percentage`
            FROM ranked_transactions tab
            LEFT JOIN db_hiring_test.raw_transactions_fee fee 
                ON CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) >= CAST(fee.`range-start` AS DECIMAL(10,2)) 
                AND CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) <= CAST(fee.`range-end` AS DECIMAL(10,2))
            WHERE tab.rn = 1 AND tab.status = 'Confirmed' AND SentDate >= '2021-02-01' AND SentDate < '2021-03-01'
        )
        SELECT AddressOrigin, SUM(FixedTotalSent * `fee-percentage` / 100) AS fee  FROM transactions
        GROUP BY AddressOrigin
        ORDER BY fee DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 3- Qual é o id da transação com a maior taxa paga?
# R: ID635

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        ),
        transactions AS
        (
            SELECT IdTransaction,
                CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) AS FixedTotalSent, 
                `fee-percentage`
            FROM ranked_transactions tab
            LEFT JOIN db_hiring_test.raw_transactions_fee fee 
                ON CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    >= CAST(fee.`range-start` AS DECIMAL(10,2)) 
                AND CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    <= CAST(fee.`range-end` AS DECIMAL(10,2))
            WHERE tab.rn = 1 AND tab.status = 'Confirmed'
        )
        SELECT IdTransaction, FixedTotalSent, `fee-percentage`, (FixedTotalSent * `fee-percentage` / 100) AS fee  FROM transactions
        ORDER BY fee  DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 4- Qual é a média de taxa paga considerando todas as transações realizadas?
# R: 21713,05.

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        ),
        transactions AS
        (
            SELECT CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) AS FixedTotalSent, 
                `fee-percentage`
            FROM ranked_transactions tab
            LEFT JOIN db_hiring_test.raw_transactions_fee fee 
                ON CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    >= CAST(fee.`range-start` AS DECIMAL(10,2)) 
                AND CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    <= CAST(fee.`range-end` AS DECIMAL(10,2))
            WHERE tab.rn = 1 AND tab.status = 'Confirmed'
        )
        SELECT AVG(FixedTotalSent * `fee-percentage` / 100) AS fee  FROM transactions
        ORDER BY fee  DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")

In [ ]:
# 4- Qual é a média de taxa paga considerando todas as transações realizadas?
# R: 21713,05.

try:
    connection = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=database
    )
    
    with connection.cursor() as cursor:
        sql = '''
        WITH ranked_transactions AS 
        (
          SELECT *, ROW_NUMBER() OVER (PARTITION BY IdTransaction ORDER BY ImportDate DESC) AS rn
          FROM db_hiring_test.raw_transactions_table
        ),
        transactions AS
        (
            SELECT CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) AS FixedTotalSent, 
                `fee-percentage`
            FROM ranked_transactions tab
            LEFT JOIN db_hiring_test.raw_transactions_fee fee 
                ON CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    >= CAST(fee.`range-start` AS DECIMAL(10,2)) 
                AND CAST(REGEXP_REPLACE(TotalSent, '[,()]+', '') AS DECIMAL(10,2)) 
                    <= CAST(fee.`range-end` AS DECIMAL(10,2))
            WHERE tab.rn = 1 AND tab.status = 'Confirmed'
        )
        SELECT AVG(FixedTotalSent * `fee-percentage` / 100) AS fee  FROM transactions
        ORDER BY fee  DESC LIMIT 1;
        '''
        cursor.execute(sql)
        result = cursor.fetchall()
        for row in result:
            print(row)

    connection.close()

except pymysql.Error as e:
    print(f"Error connecting to MySQL was: {e}")